In [1]:
import os 
os.chdir('../')


In [26]:

import torch
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score


In [4]:
emb =[]
f = open("./text_only/embeddings/bert_embeddings", "r")
s = f.readlines()
for t in s:
    t = t[1:-2]
    k = t.split(",")
    emb.append(k)
print(len(emb) == 9989)

emb1 = []
for v in emb:
    v = list(map(float, v))
    emb1.append(v)
    
print(len(emb1) == 9989)    

True
True


In [5]:
sentence_emebeddings_tensor = torch.tensor(emb1)
print(len(sentence_emebeddings_tensor[0]))

768


In [6]:
df_emo = pd.read_csv('./data/MELD/train_sent_emo.csv')

X = emb1
y = df_emo['Emotion']

In [7]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


svm = SVC()

svm.fit(X_train,y_train)

SVC()

In [16]:
y_pred = svm.predict(X_test)

score = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(score)
print(f1)

0.6002426448286321
0.5454186897915175


In [11]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)

rfc.fit(X_train, y_train)




RandomForestClassifier()

In [17]:
forest_pred = rfc.predict(X_test)
score = accuracy_score(y_test, forest_pred)
f1 = f1_score(y_test, forest_pred, average='weighted')
print(score)
print(f1)

0.5735517136791022
0.5089871736698142


In [33]:
from sklearn.neighbors import KNeighborsClassifier

def elegant_cv(XTrain, YTrain):
    '''
    This method does the same as the method above except it uses accuracy and 'cross_val_score' from sklearn
    It is the other function that is used
    credit: me on assignment 2
    '''
    kfold = KFold(n_splits=10, shuffle=True, random_state=1)
    classifiers = []
    Ks = [21,25,28,30,44,50,100]

    for k in Ks:
        classifier = KNeighborsClassifier(k)
        cv_score = cross_val_score(classifier, XTrain, YTrain, cv=kfold, scoring='accuracy')
        classifiers.append((k,cv_score.mean()))
    
    print(classifiers)
    k_best = sorted(classifiers, key =lambda x: x[1], reverse=True)[0][0] #sort the reuslts of the different Ks and pick the one with the lowest 0-1 loss
    print(k_best)
    return k_best

k_best = elegant_cv(X_train, y_train)

knc = KNeighborsClassifier(k_best)




[(21, 0.5603707917810052), (25, 0.5645530196550878), (28, 0.5657474957053299), (30, 0.5638063047988756), (44, 0.5587240925417755), (50, 0.5609662450081432), (100, 0.5514012895165428)]
28


In [34]:
knc.fit(X_train, y_train)
    
knc_pred = knc.predict(X_test)
score = accuracy_score(y_test, knc_pred)
f1 = f1_score(y_test, knc_pred, average='weighted')
print(score)
print(f1)

0.5638459205338187
0.5165775826573011
